In [ ]:
# 베이즈 회귀(bayesian_regression)

# 베이즈 통계는 오늘날 실증적 금융공학(empirical finance)의 초석이다.
# 가장 흔한 해석은 베이즈 정리에 대한 통시적(diachronic)해석이다.
# 이는 시간이 흐르면서 시계열의 평균 수익률 등 우리가 관심을 가지고 있는 어떤 변수나 모수에 대해 새로운 정보를 얻게 되는 상황을 말한다.

![bayesian_regression](./bayesian.jpg)

In [4]:
# PyMC3는 베이즈 통계를 기술적으로 구현할 수 있는 강력한 고성능의 라이브러리다.
import warnings
warnings.simplefilter('ignore')
import pymc3 as pm
import numpy as np
np.random.seed(1000)
import matplotlib.pyplot as plt
%matplotlib inline

# 기초예제 시각화
x = np.linspace(0, 10, 500)
y = 4 + 2 * x + np.random.standard_normal(len(x)) * 2

# 벤치마크로 잡음이 있는 자료를 최소 자승 회귀분석한다.
reg = np.polyfit(x, y, 1)   # linear regression

# 시각화해보자
plt.figure(figsize=(8, 4))
plt.scatter(x, y, c=y, marker='v')
plt.plot(x, reg[1] + reg[0] * x, lw=2.0)
plt.colorbar()
plt.grid(True)
plt.xlabel('x')
plt.ylabel('y')
# tag: pm_fig_0
# title: Sample data points a regresion line
# size: 90


You can find the C code in this temporary file: C:\Users\inventor\AppData\Local\Temp\theano_compilation_error__vti4qyy


AttributeError: module 'theano' has no attribute 'gof'

In [ ]:
# 표준 회귀 방식으로 구한 결과는 다음과 같이 고정된 회귀선의 인수 값이다

print(reg)

# 결과에서 최고차 모수 값(이 경우에는 회귀선의 기울기)의 인덱스는 0이고 절편의 인덱스가 1이다.
# 자료에 포함된 잡음의 영향으로 원래의 인수 값인 2와 4가 완벽하게 복원되지는 못했다.

# 다음으로 베이즈 회귀분석을 한다. 우선 모수가 특정한 분포를 가진다고 가정해야 한다.
# 예를 들어 회귀선이 수식 y(x) = a + bx 로 나타나는 경우 각 모수는 다음과 같은 분포를 가진다고 가정하자
# - a는 평균이 0이고, 표준편차가 20인 정규분포이다.
# - b는 평균이 0이고, 표준편차가 20인 정규분포이다.
# 우도(likelihood)의 경우 평균이 y(x)이고 정규분포이며 그 표준편차는 0부터 10시아에서 상수 분포를 가지는 확률변수라고 가정한다.
# 베이즈 회귀분석의 중요한 방법 중의 하나는 마코프 체인 몬테카를로(MCMC)방법이다.
# MCMC는 좀 더 구조적이고 자동화되어 있을 뿐 그 원리 상으로 상자에서 공을 꺼내는 앞의 예제와 다를 바 없다.


# 몬테카를로 샘플링을 위한 세가지 다른 함수가 제공된다.
# (1) find_MAP 함수는 사후 확률을 최대화하는 국지점(local maximum a posteriori point)을 계산하여 샘플링 알고리즘의 시작점을 찾는다.
# (2) NUTS 함수는 efficient No-U-Turn Sampler with dual averaging의 약자로 주어진 사전 확률에 따른 MCMC 샘플링 알고리즘을 구현한다.
# (3) sample 함수는 find_MAP에서 주어진 초깃값과 NUTS 알고리즘에서 구한 최적 구간 크기를 사용하여 샘플을 생성한다.

# 이 함수들은 PyMC3 모형 객체에 포함되어 있으므로 다음과 같이 with 문장으로 실행할 수 있다.

with pm.Model() as model: 
        # PyMC3의 모형은 with문 안에서 사용된다.
    # 사전 확률 정의
    alpha = pm.Normal('alpha', mu=0, sd=20)
    beta = pm.Normal('beta', mu=0, sd=20)
    sigma = pm.Uniform('sigma', lower=0, upper=10)
    
    # 선형 회귀선 정의
    y_est = alpha + beta * x
    
    # 우도 정의
    likelihood = pm.Normal('y', mu=y_est, sd=sigma, observed=y)
    
    # 추정 과정
    start = pm.find_MAP()
      # 최적화를 사용하여 시작값 추정
    step = pm.NUTS()
      # MCMC 샘플링 알고리즘 인스턴스 생성
    trace = pm.sample(100, step, start=start, progressbar=False)
      # NUTS 샘플링을 사용한 100개의 사후 샘플 생성

In [ ]:
# 첫 번째 샘플에서 추정한 값을 보자
print(trace[0])

# 세 값이 모두 원래의 값(4,2,2)와 유사하다. 물론 전체 과정 동안 더 많은 추정값을 생성한다.
# traceplot을 사용하면 결과를 더 잘 설명할 수 있다.
# traceplot은 모든 모수에 대해 최종 사후 분포와 샘플마다의 추정값을 보여준다.
fig = pm.traceplot(trace, lines={'alpha': 4, 'beta': 2, 'sigma': 2})
plt.figure(figsize=(8, 8))
# tag: pm_fig_1
# title: Trace plots for alpha, beta and sigma
# size: 90

In [ ]:
# 회귀분석 결과에서 alpha, beat 값만 사용하여 회귀선을 그릴 수 있다.
plt.figure(figsize=(8, 4))
plt.scatter(x, y, c=y, marker='v')
plt.colorbar()
plt.grid(True)
plt.xlabel('x')
plt.ylabel('y')
for i in range(len(trace)):
    plt.plot(x, trace['alpha'][i] + trace['beta'][i] * x)
# tag: pm_fig_2
# title: Sample data and regression lines from Bayesian regression
# size: 90

In [ ]:
# 가상의 자료에 대해 PyMC3를 사용하여 베이즈 회귀분석을 해보았다. 이제는 현실의 시장 자료로 옮겨간다.
import pytz
import datetime as dt
import os

path = os.getcwd()

os.chdir(path)

raw = pd.read_csv('./source/tr_eikon_eod_data.csv',
                  index_col=0, parse_dates=True)
symbols = ['GLD', 'GDX'] # 금 원자재 etf와 금광 회사 주식 etf
# GLD : SPDR골드셰어즈
# GDX : VANECK VECTORS/GOLD MINERS ETF
data = raw[symbols]

print(data.info())

# 두 etf의 역사적인 가격을 표시하자
data.plot(figsize=(8, 4))
# tag: zip_fig_1
# title: Co-movements of traiding pair
# size: 90

In [ ]:
# 두 etf의 절대 수익률은 크게 다르다
print(data.iloc[-1] / data.iloc[0] - 1)

# 하지만 두 시계열은 강한 양의 상관관계를 가지고 있다.
print(data.corr())

# matplotlib에서 날짜 시간 정보를 사용하려면 다음과 같이 날짜 표현을 변경해야한다.

import matplotlib as mpl
mpl_dates = mpl.dates.date2num(data.index.to_pydatetime())
print(mpl_dates)

#GLD와 GDX의 가격을 스캐터 플롯에 나타내고 날짜에 따라 스캐터 플롯의 점 색깔을 다르게 표시하자

plt.figure(figsize=(8, 4))
plt.scatter(data['GDX'], data['GLD'], c=mpl_dates, marker='o')
plt.grid(True)
plt.xlabel('GDX')
plt.ylabel('GLD')
plt.colorbar(ticks=mpl.dates.DayLocator(interval=250),
             format=mpl.dates.DateFormatter('%d %b %y'))
# tag: zip_fig_2
# title: Scatter plot of prices for GLD and GDX
# size: 90


In [ ]:
# 이 두 시계열의 베이시스에 대해 베이즈 회귀분석을 수행한다. 모수 모형은 이전의 가상 자료의 경우와 같다.
# 다만 가상 자료를 현실 자료로 바꾼다.

with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sd=20)
    beta = pm.Normal('beta', mu=0, sd=20)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    
    y_est = alpha + beta * data['GDX'].values
    
    likelihood = pm.Normal('GLD', mu=y_est, sd=sigma,
                           observed=data['GLD'].values)
    
    start = pm.find_MAP()
    step = pm.NUTS()
    trace = pm.sample(100, step, start=start, progressbar=False)

# 주어진 세 가지 모수의 사전 확률 분포에 대해 MCMC 샘플링한 결과를 나타내었다.
# GLD와 GDX 자료에 기반한 alpha, beta, sigma 모수의 traceplot
fig = pm.traceplot(trace)
plt.figure(figsize=(8, 8))
# tag: zip_fig_3
# title: Trace plots for alpha, beta and sigma based on GDX and GLD data
# size: 90

In [ ]:
# 아까의 스캐터 플롯에 분석 결과로 얻은 회귀선을 추가하였다. 모든 회귀선들이 서로 가까이 있다.
plt.figure(figsize=(8, 4))
plt.scatter(data['GDX'], data['GLD'], c=mpl_dates, marker='o')
plt.grid(True)
plt.xlabel('GDX')
plt.ylabel('GLD')
for i in range(len(trace)):
    plt.plot(data['GDX'], trace['alpha'][i] + trace['beta'][i] * data['GDX'])
plt.colorbar(ticks=mpl.dates.DayLocator(interval=250),
             format=mpl.dates.DateFormatter('%d %b %y'))
# tag: zip_fig_4
# title: Scatter plot with "simple" regression lines
# size: 90

In [ ]:
# 그림에서 보듯이 방금 사용한 회귀 방법에는 큰 단점이 있다.
# 시간에 따른 변화를 고려하지 못한다는 점이다. 즉 최근의 자료를 과거의 자료와 동일하게 다루고 있다.
# 베이즈 방법은 일반적으로 통시적이다. 즉 시간이 지나면서 새로운 자료가 추가되고 회귀 및 추정 결과가 개선된다.

# 이러한 개념을 적용하려면 회귀 모수가 고정 확률 분포가 아니라 시간에 따라 변화하는 일종의 랜덤 워크 형태이어야 한다.
# 금융공학 이론에서 확률 변수로부터 확률 과정(확률 변수의 순열)으로 바뀌는 과정과 같은 형태의 일반화이다.

# 새로운 PyMC3 모형을 정의한다. 이번에는 모수를 랜덤 워크로 설정한다. 그 분산값은(샘플링 특성 개선을 위해) 로그 공간으로 변환하여 사용한다.

from pymc3.distributions import Exponential

model_randomwalk = pm.Model()
with model_randomwalk:
    sigma_alpha = Exponential('sigma_alpha', 1. / .02, testval=.1)
    sigma_beta = Exponential('sigma_beta', 1. / .02, testval=.1)


# 모수가 랜덤 워크 과정이라고 가정하였으므로 다음과 같이 alpha와 beta에 대해 랜덤 워크 가정을 적용한다.
# 전체 계산 과정의 효율성을 위해 50개 자료씩 같은 계수를 공유하도록 한다.

from pymc3.distributions.timeseries import GaussianRandomWalk

subsample_alpha = 50
subsample_beta = 50

with model_randomwalk:
    alpha = GaussianRandomWalk('alpha', sigma_alpha**-2, shape=len(data) // subsample_alpha)
    beta = GaussianRandomWalk('beta', sigma_beta**-2, shape=len(data) // subsample_beta)
    alpha_r = np.repeat(alpha, subsample_alpha)
    beta_r = np.repeat(beta, subsample_beta)

# 시계열 자료의 길이
print(len(data.dropna().GDX.values))


In [ ]:
# 50개씩 나누어 적용하기 위해 전체 자료 중 앞 부분의 1950개 자료만 회귀분석에 사용한다.
with model_randomwalk:
    regression = alpha_r + beta_r * data.GDX.values[:1950]
    sd = pm.Uniform('sd', 0, 20)
    likelihood = pm.Normal('GLD', 
                           mu=regression, 
                           sd=sd, 
                           observed=data.GLD.values[:1950])

# 단일 확률 변수가 아닌 랜덤 워크 과정을 사용하였기 때문에 모형 정의가 더 복잡해졌다.
# 그러나 MCMC를 사용한 추정 과정은 본질적으로 이전과 동일하다.
# 전과 같이 한 번만 모수를 추정하는 것이 아니고 1,950/50 = 39번의 추정 과정이 있기 때문에 계산 부담이 급격히 증가했다.
import scipy.optimize as sco
with model_randomwalk:
    start = pm.find_MAP(vars=[alpha, beta], fmin=sco.fmin_l_bfgs_b)
    step = pm.NUTS(scaling=start)
    trace_rw = pm.sample(100, step, start=start, progressbar=False)

# 모두 39개의 시간 구간에 대해 100개의 추정값이 나왔다.
print(np.shape(trace_rw['alpha']))


In [ ]:
# 회귀분석 결과인 alpha와 beta 모수 값이 시간에 따라 변해가는 것을 추정치 일부와 평균값을 사용하여 시각화하자
part_dates = np.linspace(min(mpl_dates), max(mpl_dates), 39)

fig, ax1 = plt.subplots(figsize=(10, 7))
plt.plot(part_dates, np.mean(trace_rw['alpha'], axis=0), 'b', lw=2.5, label='alpha')
for i in range(55):
    plt.plot(part_dates, trace_rw['alpha'][i], 'b-.', lw=0.75)
plt.xlabel('date')
plt.ylabel('alpha')
plt.axis('tight')
plt.grid(True)
plt.legend(loc=2)
ax1.xaxis.set_major_formatter(mpl.dates.DateFormatter('%d %b %y') )
ax2 = ax1.twinx()
plt.plot(part_dates, np.mean(trace_rw['beta'], axis=0), 'r', lw=2.5, label='beta')
for i in range(55):
    plt.plot(part_dates, trace_rw['beta'][i], 'r-.', lw=0.75)
plt.ylabel('beta')
plt.legend(loc=4)
fig.autofmt_xdate()
plt.show()

# (평균)alpha와 (평균)beta 추정값이 시간에 따라 변하는 모습
# 주성분 분석을 구현해던 때와 마찬가지로 베이즈 통계 예제에서도 상대적인(로그) 수익률이 아닌 절대적인 가격 수준을 사용하였다.
# 이러한 방식은 단지 설명을 위한 것으로 그래프 결과가 보기 쉽고 시각적으로 이해 하기 쉽기 때문이다.
# 실제 금융 응용에서는 상대적인 수익률 자료를 사용해야 한다.


In [ ]:
# 평균 alpha, beta 값을 써서 회귀분석 결과가 시간에 따라 어떻게 변하는지 보였다.
# 스캐터 플롯을 다시 시각화하여 평균 alpha, beta 값을 사용한 39개의 회귀선을 추가하였다.
# 시간이 지나면서 회귀 결과가 최신의 자료에 맞도록 생신되고 있음을 볼 수 있다.
# 즉, 각 시간 구간은 그 구간만의 회귀 결과를 가진다.

plt.figure(figsize=(10,7))
plt.scatter(data['GDX'], data['GLD'], c=mpl_dates, marker='o', cmap=mpl.cm.jet)
plt.colorbar(ticks=mpl.dates.DayLocator(interval=250),
             format=mpl.dates.DateFormatter('%d %b %y'))
plt.grid(True)
plt.xlabel('GDX')
plt.ylabel('GLD')
x = np.linspace(min(data['GDX']), max(data['GDX'])) 
for i in range(39):
    alpha_rw = np.mean(trace_rw['alpha'].T[i])
    beta_rw = np.mean(trace_rw['beta'].T[i]) 
    plt.plot(x, alpha_rw + beta_rw * x, alpha=0.3,
             color=plt.cm.jet(256 * i // 39))
plt.show()    

# 여기까지 베이즈 화귀분석과 파이썬이 PyMC3 라이브러리로 베이즈 통계를 어떻게 구현하는지를 보여준다
# 베이즈 회귀분석은 최근 정략적 금융 분석에서 아주 중요하고 인기 있는 도구의 하나가 되고 있다.

